In [1]:
import torch
import pickle
import numpy as np
import mmpretrain
import mmengine
from mmengine.fileio import get
from mmpretrain.apis import init_model
from mmengine.device import get_device
import mmengine
from mmengine.config import Config, ConfigDict, DictAction
from mmengine.evaluator import DumpResults
from mmengine.registry import RUNNERS
from mmengine.runner import Runner
torch.cuda.empty_cache()

In [2]:
path = '/scratch/yw6594/cf/data/cifar10/cifar-10-batches-py/cifar_test_nolabels.pkl'
data = mmengine.load(path)
imgs = data[b'data']
config = '/scratch/yw6594/cf/mmcl/cifar-img/dl_res18_exp/cifar10_resnet18_c32_kaggle.py'
model_cpt = '/scratch/yw6594/cf/mmcl/out/dl/res18-c10/cifarbase/c32/epoch_200.pth'


model = init_model(config, model_cpt)
# cfg = Config.fromfile(config)
# cfg.load_from = model_cpt
# runner = Runner.from_cfg(cfg)



Loads checkpoint by local backend from path: /scratch/yw6594/cf/mmcl/out/dl/res18-c10/cifarbase/c32/epoch_200.pth


In [3]:
### data ..to(get_device())
output = []
length = len(imgs)
bs = 1000
with torch.no_grad():
    model = model.to(get_device())
    mean = torch.tensor([125.307, 122.961, 113.8575]).view(-1, 1, 1).to(get_device()).detach()
    std = torch.tensor([51.5865, 50.847, 51.255]).view(-1, 1, 1).to(get_device()).detach()
    for i, img in enumerate(imgs):
        img = img.reshape(-1, 3, 32, 32)
        img = torch.tensor(img, dtype=torch.float).to(get_device())
        img = (img - mean)/std
        pred = model(img).detach()
        output.append({"ID":i,"Labels":pred.cpu().max(1)[1].item()})
        del img, pred
    del model
torch.cuda.empty_cache()
# print(output)
# output = torch.cat(output,dim=0)

In [ ]:
torch.cuda.empty_cache()
import pandas as pd
df = pd.DataFrame(output)
df.to_csv('/scratch/yw6594/cf/mmcl/out/dl/res18-c10/out_32.csv', index=False)